In [221]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [222]:
df = pd.read_csv('lifeexpectancy.csv')

In [223]:
df.head()

,Country,Year,Status,Adult_Mortality,BMI,Alcohol,HIV_AIDS,Total_expenditure,Income_composition_of_resources,Schooling,Life_expectancy
0,Canada,2024,Developed,91,23.019938,4.499131,1.080157,7.845313,0.542585,12.046433,90.00000
1,Germany,2005,Developed,215,21.950533,8.941229,4.836356,6.703710,0.620611,16.947498,90.00000
2,France,2025,Developed,131,22.777076,6.507659,3.575625,14.811973,0.676434,16.698824,90.00000
3,Brazil,2011,Developing,340,31.659207,2.466083,4.785674,1.782969,0.422900,10.826091,89.37576
4,Canada,2022,Developed,207,NaN,NaN,2.135525,2.990539,0.885590,8.567364,90.00000


In [224]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          1200 non-null   object 
 1   Year                             1200 non-null   int64  
 2   Status                           1200 non-null   object 
 3   Adult_Mortality                  1200 non-null   int64  
 4   BMI                              1080 non-null   float64
 5   Alcohol                          1080 non-null   float64
 6   HIV_AIDS                         1080 non-null   float64
 7   Total_expenditure                1200 non-null   float64
 8   Income_composition_of_resources  1080 non-null   float64
 9   Schooling                        1080 non-null   float64
 10  Life_expectancy                  1200 non-null   float64
dtypes: float64(7), int64(2), object(2)
memory usage: 103.3+ KB


In [225]:
df.shape

(1200, 11)

***Data Cleaning***

In [226]:
#year column dropped
df.drop(columns=['Year'],inplace=True)
df.head()

,Country,Status,Adult_Mortality,BMI,Alcohol,HIV_AIDS,Total_expenditure,Income_composition_of_resources,Schooling,Life_expectancy
0,Canada,Developed,91,23.019938,4.499131,1.080157,7.845313,0.542585,12.046433,90.00000
1,Germany,Developed,215,21.950533,8.941229,4.836356,6.703710,0.620611,16.947498,90.00000
2,France,Developed,131,22.777076,6.507659,3.575625,14.811973,0.676434,16.698824,90.00000
3,Brazil,Developing,340,31.659207,2.466083,4.785674,1.782969,0.422900,10.826091,89.37576
4,Canada,Developed,207,NaN,NaN,2.135525,2.990539,0.885590,8.567364,90.00000


In [227]:
misscol = ['BMI','Alcohol','HIV_AIDS','Total_expenditure','Income_composition_of_resources','Schooling']

In [228]:
df.isnull().sum()

Country                              0
Status                               0
Adult_Mortality                      0
BMI                                120
Alcohol                            120
HIV_AIDS                           120
Total_expenditure                    0
Income_composition_of_resources    120
Schooling                          120
Life_expectancy                      0
dtype: int64

#### Training model 

In [229]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing  import OneHotEncoder,StandardScaler,OrdinalEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [230]:
x = df.drop(columns=['Life_expectancy'],axis=1)
y = df['Life_expectancy']

In [231]:
# train test spilt
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42,test_size=0.2)

In [232]:
x_train.shape , x_test.shape

((960, 9), (240, 9))

#### Pre Processing


In [233]:
numeric_col = ['Adult_Mortality','BMI','Alcohol','HIV_AIDS','Total_expenditure','Income_composition_of_resources','Schooling']
categor_col = ['Country']
ordinal_col = ['Status']

In [234]:
# impute + scale
num_transf = Pipeline([
    ('impute',SimpleImputer(strategy='mean')),
    ("scaling",StandardScaler())
])

In [235]:
ord_tranf = Pipeline([
    ("ordinal", OrdinalEncoder(categories=[['Developed','Developing']])),
])

In [236]:
ohe_tranf = Pipeline([
    ('onehot',OneHotEncoder(drop='first',handle_unknown='ignore'))
])

In [237]:
preprocessing = ColumnTransformer([
    ('num',num_transf,numeric_col),
    ('ord',ord_tranf,ordinal_col),
    ('cat',ohe_tranf,categor_col),
])

In [238]:
mypipe = Pipeline([
    ("preprocessor",preprocessing),
    ("model",LinearRegression()),
])

In [239]:
pipeline  = mypipe.fit(x_train,y_train)

In [240]:
ypred = pipeline.predict(x_test)

In [241]:
r2 = r2_score(y_test,ypred)
print(r2)

0.11561643434390934


In [246]:
mse = mean_squared_error(y_test, ypred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, ypred)

print(f"R² Score: {r2_score(y_test, ypred):.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")

R² Score: 0.116
RMSE: 1.094
MAE: 0.353
